<a href="https://colab.research.google.com/github/SEEsuite/colab_scripts/blob/main/Owl_object_detection_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Workshop - Writing queries for joint models. 
[model](https://huggingface.co/google/owlvit-base-patch32) | [paper](https://arxiv.org/pdf/2205.06230.pdf) 

The model has been taught the relationship between words and images (limited, to be sure). It is a large model so it will run slowly, and colab may give you some pushback. When in doubt, restart the session.

We pull images from instagram using a temporary url. this url has to be regenerated every week, unfortunately. I'll provide you with a separate script to do that.

This program will return labels, bounding boxes, and scores for each image post . I still don't think this format is going to be incredibly helpful - I'll look a better way to save the images so that you can instantly click on the image in concern.

Please enter labels into the text variable, formatting them in natural language.


In [ ]:
## CHANGE THIS
link = "https://docs.google.com/spreadsheets/d/1UZkl8R0963yWcGxno0RZ0DBW0sEx_zZD/edit?usp=sharing&ouid=101042095541764641159&rtpof=true&sd=true"
#THE SCRIPT EXPECTS A COLUMN CALLED 'temporary_urls' CONTAINGIN URL TO IMAGE

Query Images.






# Use on Your Own Data

In [ ]:
!pip install transformers
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 24.6 MB/s eta 0:00:0

In [ ]:
# importing miscelaneaous packages 
import numpy as np # fast manipulation of multidimensional arrays
from numpy import mean
import pandas as pd
import math

from tqdm.notebook import tqdm as progress_bar # a little vizualization of how fast a loop is running
from scipy.special import softmax
import csv

#dealing with images
from PIL import Image
from PIL import Image, ImageDraw

# helps with API request
import requests

# help with deep learning
from datasets import Dataset
import torch

In [ ]:
import urllib.request
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

def import_data_from_drive(share_link, your_name_for_file="my_data"):
  """Brings data file from a google drive sharepoint to your colab workspace.
     It does not require you to host the dataset on your own account.

     Parameters:
     share_link: the link to view a file in google drive
     our_name_for_file: a string describing the file, preferable endling in a file type, ex. 'data.csv'
     """
  id = share_link.split("/")[5] # separate the id from the link
  print("Using id", id, "to find file on drive")

  # use pydrive and colab modules to authenticate you
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  print("Authenticated colab user")

  # This step will move the file from Drive to the workspace
  download_object = drive.CreateFile({'id':id}) 
  download_object.GetContentFile(your_name_for_file)
  print("Added file to workspace with name", your_name_for_file)

  return

In [ ]:

import_data_from_drive(link, your_name_for_file="insta.xlsx")
df = pd.read_excel('insta.xlsx')

Using id 1UZkl8R0963yWcGxno0RZ0DBW0sEx_zZD to find file on drive
Authenticated colab user
Added file to workspace with name insta.xlsx


In [ ]:
df

,Query Id,Query Name,Date,Title,Snippet,Full Text,Url,Domain,Sentiment,Emotion,...,Reach (new),Reddit Score,Reddit Score Upvote Ratio,Reddit Author Karma,Reddit Author Awardee Karma,Reddit Author Awarder Karma,Reddit Comments,Subreddit,Subreddit Subscribers,temporary_urls
0,2001569258,Sponsored IG test,2023-01-01 00:50:40.000,💙🖤 #explorepage #blackgaymen #face #lgbt #mode...,💙🖤 #explorepage #blackgaymen #face #lgbt #mode...,💙🖤 #explorepage #blackgaymen #face #lgbt #mode...,https://www.instagram.com/p/Cm2mIrYrUJb/,instagram.com,neutral,Joy,...,380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['https://scontent-dfw5-1.cdninstagram.com/v/t...
1,2001569258,Sponsored IG test,2022-12-31 20:37:05.000,Stay tuned for our future episodes!! Here’s ou...,Stay tuned for our future episodes!! Here’s ou...,Stay tuned for our future episodes!! Here’s ou...,https://www.instagram.com/p/Cm2JHTGPSKT/,instagram.com,neutral,Joy,...,949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['https://scontent-dfw5-2.cdninstagram.com/v/t...
2,2001569258,Sponsored IG test,2022-12-31 20:37:05.000,Stay tuned for our future episodes!! Here’s ou...,Stay tuned for our future episodes!! Here’s ou...,Stay tuned for our future episodes!! Here’s ou...,https://www.instagram.com/p/Cm2JHTGPSKT/,instagram.com,neutral,Joy,...,949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['https://scontent-dfw5-2.cdninstagram.com/v/t...
3,2001569258,Sponsored IG test,2022-12-31 20:37:05.000,Stay tuned for our future episodes!! Here’s ou...,Stay tuned for our future episodes!! Here’s ou...,Stay tuned for our future episodes!! Here’s ou...,https://www.instagram.com/p/Cm2JHTGPSKT/,instagram.com,neutral,Joy,...,949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['https://scontent-dfw5-2.cdninstagram.com/v/t...
4,2001569258,Sponsored IG test,2022-12-31 20:37:05.000,Stay tuned for our future episodes!! Here’s ou...,Stay tuned for our future episodes!! Here’s ou...,Stay tuned for our future episodes!! Here’s ou...,https://www.instagram.com/p/Cm2JHTGPSKT/,instagram.com,neutral,Joy,...,949,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['https://scontent-dfw5-1.cdninstagram.com/v/t...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1547,2001569258,Sponsored IG test,2022-01-02 19:47:14.000,have the holidays stressed you out? are you si...,...#sundayscaries #goth #gothgoth #sponsored #...,have the holidays stressed you out? are you si...,https://www.instagram.com/p/CYPW2pDvrOe/,instagram.com,negative,NaN,...,4081,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['https://scontent-dfw5-2.cdninstagram.com/v/t...
1548,2001569258,Sponsored IG test,2022-01-02 11:43:01.000,A volte sono anche vestito 😌🤜🏻 - - - - - - - -...,A volte sono anche vestito 😌🤜🏻 - - - - - - - -...,A volte sono anche vestito 😌🤜🏻 - - - - - - - -...,https://www.instagram.com/p/CYOfcGLs8__/,instagram.com,neutral,NaN,...,4304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['https://scontent-dfw5-2.cdninstagram.com/v/t...
1549,2001569258,Sponsored IG test,2022-01-02 04:57:29.000,Come to my aid babies! 😢 I'm waiting for you i...,...#lgbt #following #toesucking #americanguys ...,Come to my aid babies! 😢 I'm waiting for you i...,https://www.instagram.com/p/CYNxB2XKj7-/,instagram.com,positive,Sadness,...,574,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['https://scontent-dfw5-1.cdninstagram.com/v/t...
1550,2001569258,Sponsored IG test,2022-01-02 03:51:47.000,This New Years hangover is #sponsored by heyya...,This New Years hangover is #sponsored by heyya...,This New Years hangover is #sponsored by heyya...,https://www.instagram.com/p/CYNpgrzLiSN/,instagram.com,neutral,NaN,...,2399,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['https://scontent-dfw5-2.cdninstagram.com/v/t...


In [ ]:
### This will reformat the urls if they are getting stored funny. (I think saving the df is messing with them sometimes)
def app(url):
  return url.strip(']\'[').split(', ')

df["temporary_urls"] = df["temporary_urls"].apply(app)

In [ ]:
def to_pillow(batch):
    """ a transformation to be applyed to a dataset. takes in a url string and returns data with a new column"""
    images = []
    for url_list in batch['temporary_urls']:
      # print(url_list)
      for url in url_list: # I need to get this running for multiple images per post
        # print(url)
        try:
          http_image = requests.get(url, stream=True).raw #1
          image = Image.open(http_image) # step 2
          # print(type(http_image))
          # image.show()
          # print(type(image))
        except Exception as e:
          print(e)
          image = None
        images.append(image)
    batch['images'] = images # will probably need to expand this out in practice
    return batch

In [ ]:
dataset = Dataset.from_pandas(df)
dataset.set_transform(to_pillow)

In [ ]:
df['temporary_urls'][0][0]

'https://scontent-dfw5-1.cdninstagram.com/v/t51.29350-15/323568938_211520117902124_3209011309299629821_n.jpg?stp=dst-jpg_s640x640&_nc_cat=105&ccb=1-7&_nc_sid=8ae9d6&_nc_ohc=kvYJ3F0tLO8AX8nS0q3&_nc_ht=scontent-dfw5-1.cdninstagram.com&oh=00_AfDkHP_v8_wcgreqFNABAiW4bq-YmEavp6iZCpxu-0KMtA&oe=643A5968'

In [ ]:
from transformers import OwlViTProcessor, OwlViTForObjectDetection

processor = OwlViTProcessor.from_pretrained("google/owlvit-base-patch32")
model = OwlViTForObjectDetection.from_pretrained("google/owlvit-base-patch32")

In [ ]:
#Just copy and pasted from image_analysis_1!
predictions = pd.DataFrame(columns=["post_url", 'query_match', 'score', 'bounding_box','total_size'])
model.to('cuda')

# veiw the image live. Will weigh colab down - I need to look into this. 
print_image = False

for data in progress_bar(dataset): # change 1

  #pass images through the preprocessor
  texts = [[ "a cat", "a dog", "a closeup photo of a persons face", "a book", "a poster", "a group of people laughing together", "a bench", "a cup", "a cup held in someone's hand",  "a hand with manicured nails"]]
  images = data['images']

  # print(len(image))

  if images:
    inputs = processor(text=texts,images=images, return_tensors="pt").to('cuda')# change 2
  else:
    continue

  with torch.no_grad():
    #TODO is resize neccesary?
    if print_image:
      images.show()
    outputs = model(**inputs)

    # Target image sizes (height, width) to rescale box predictions [batch_size, 2]
    target_sizes = torch.Tensor([images.size[::-1]]).to('cuda')
    # Convert outputs (bounding boxes and class logits) to COCO API
    results = processor.post_process(outputs=outputs, target_sizes=target_sizes)

    i = 0  # Retrieve predictions for the first image for the corresponding text queries
    text = texts[i]
    boxes, scores, labels = results[i]["boxes"], results[i]["scores"], results[i]["labels"]

    # Print detected objects and rescaled box coordinates
    score_threshold = 0.15
    for box, score, label in zip(boxes, scores, labels):
      box = [round(i, 2) for i in box.tolist()]
      if score >= score_threshold:
          print(f"Detected {text[label]} with confidence {round(score.item(), 3)} at location {box}")

          if print_image:
            img1 = ImageDraw.Draw(images)  
            img1.rectangle(box, outline ="green")
            images.show()

          # save these
          pred_row = pd.DataFrame(columns=["post_url", 'query_match', 'score', 'bounding_box', 'total_size'])
          pred_row["post_url"] = [data['Url']]
          pred_row["query_match"] = [text[label]]
          pred_row["score"] = [score.item()]
          pred_row.at[0, "bounding_box"] = box
          pred_row.at[0, "total_size"] = images.size
          print(images.size)

          predictions = pd.concat([predictions, pred_row], axis=0)

    if print_image:
      break
   


  0%|          | 0/1552 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/transformers/models/owlvit/image_processing_owlvit.py:355: FutureWarning: `post_process` is deprecated and will be removed in v5 of Transformers, please use `post_process_object_detection`
  warnings.warn(


Detected a closeup photo of a persons face with neutral expression with confidence 0.465 at location [-0.46, 2.05, 509.36, 515.51]
(512, 640)


/usr/local/lib/python3.9/dist-packages/transformers/models/owlvit/image_processing_owlvit.py:355: FutureWarning: `post_process` is deprecated and will be removed in v5 of Transformers, please use `post_process_object_detection`
  warnings.warn(


Detected a poster with confidence 0.399 at location [47.67, 7.86, 468.45, 640.99]
(512, 640)


...

/usr/local/lib/python3.9/dist-packages/transformers/models/owlvit/image_processing_owlvit.py:355: FutureWarning: `post_process` is deprecated and will be removed in v5 of Transformers, please use `post_process_object_detection`
  warnings.warn(


Detected a closeup photo of a persons face with neutral expression with confidence 0.204 at location [227.64, 15.76, 289.73, 103.8]
(512, 640)


/usr/local/lib/python3.9/dist-packages/transformers/models/owlvit/image_processing_owlvit.py:355: FutureWarning: `post_process` is deprecated and will be removed in v5 of Transformers, please use `post_process_object_detection`
  warnings.warn(


Detected a closeup photo of a persons face with neutral expression with confidence 0.537 at location [135.0, 72.04, 279.27, 302.69]
(512, 640)
Detected a cup with confidence 0.235 at location [387.72, 286.42, 479.04, 435.47]
(512, 640)
Detected a closeup photo of a persons face with neutral expression with confidence 0.296 at location [205.99, 40.29, 305.48, 180.09]
(572, 640)


/usr/local/lib/python3.9/dist-packages/transformers/models/owlvit/image_processing_owlvit.py:355: FutureWarning: `post_process` is deprecated and will be removed in v5 of Transformers, please use `post_process_object_detection`
  warnings.warn(


In [ ]:
#This data is mostly going to be helpful getting summary stats for the entire dataset. 
predictions

,post_url,query_match,score,bounding_box,total_size
0,https://www.instagram.com/p/Cm2mIrYrUJb/,a closeup photo of a persons face with neutral...,0.464944,"[-0.46, 2.05, 509.36, 515.51]","(512, 640)"
0,https://www.instagram.com/p/Cm2JHTGPSKT/,a poster,0.399357,"[47.67, 7.86, 468.45, 640.99]","(512, 640)"
0,https://www.instagram.com/p/Cm2JHTGPSKT/,a poster,0.193085,"[-2.49, 5.17, 507.54, 639.7]","(512, 640)"
0,https://www.instagram.com/p/Cm2JHTGPSKT/,a poster,0.578113,"[47.61, -1.52, 467.58, 631.08]","(512, 640)"
0,https://www.instagram.com/p/Cm2JHTGPSKT/,a poster,0.168089,"[-2.32, 4.92, 507.5, 639.89]","(512, 640)"
...,...,...,...,...,...
0,https://www.instagram.com/p/CYOfcGLs8__/,a closeup photo of a persons face with neutral...,0.668360,"[149.21, 73.2, 399.57, 442.22]","(512, 640)"
0,https://www.instagram.com/p/CYNxB2XKj7-/,a closeup photo of a persons face with neutral...,0.204322,"[227.64, 15.76, 289.73, 103.8]","(512, 640)"
0,https://www.instagram.com/p/CYNpgrzLiSN/,a closeup photo of a persons face with neutral...,0.537312,"[135.0, 72.04, 279.27, 302.69]","(512, 640)"
0,https://www.instagram.com/p/CYNpgrzLiSN/,a cup,0.234721,"[387.72, 286.42, 479.04, 435.47]","(512, 640)"


In [ ]:
predictions.to_excel("posts_with_query_matches.xlsx")